In [0]:
!pip install keras
!pip install h5py

In [0]:
from google.colab import drive
drive.mount('/content/drive')
root_path = 'drive/My Drive/Final_year_project/Final_Project_Notes/'

In [0]:
#check whether you are using GPU
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
!pip install -U coremltools

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model,load_model, Sequential
from keras.layers import Dense,GlobalAveragePooling2D,Input, Conv2D, MaxPooling2D, Flatten, Dropout, InputLayer
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adadelta, RMSprop
from keras.callbacks import ModelCheckpoint
import keras
import math, os, sys
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
def get_resnet():
  
    input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

    # create the base pre-trained model
    base_model = ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=False)

    for layer in base_model.layers:
        layer.trainable=False

    x = base_model.output
    x = keras.layers.Flatten()(x)
    #x = GlobalAveragePooling2D(data_format='channels_last')(x)
    #x = Dense(num_classes, activation='softmax')(x)

    updatedModel = Model(base_model.input, x)

    return  updatedModel

In [0]:
def compile_model(compiledModel):

    compiledModel.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])

In [0]:
from sklearn.utils import class_weight
import numpy as np

In [0]:
def modelFitGenerator(fitModel):

    image_size = (224, 224)
    train_data_dir = '/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Small/Train'
    validation_data_dir = '/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Small/Val'
    nb_epoch = 1
    batch_size = 64
    num_classes = 5

    num_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

    num_train_steps = math.floor(num_train_samples/batch_size)
    num_valid_steps = math.floor(num_valid_samples/batch_size)
    
    train_datagen = ImageDataGenerator(  
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

    test_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=image_size ,
      batch_size=batch_size,
      class_mode='categorical', shuffle=True
    )

    validation_generator = test_datagen.flow_from_directory(
      validation_data_dir,
      target_size=image_size ,
      batch_size=batch_size,
      class_mode='categorical', shuffle=True
    )

    filepath = "Custom_Resnet_Small.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

    print("start history model")
    history = fitModel.fit_generator(
      train_generator,
      steps_per_epoch=num_train_steps,
      epochs=nb_epoch,
      validation_data=validation_generator,
      validation_steps=num_valid_steps,
       class_weight=class_weights)



In [0]:
def main():
    
    model = Sequential()
    model.add(get_resnet())
    model.add(Dense(512, activation='relu', input_dim=(224, 224,3)))
    model.add(Dropout(0.6))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
    
    modelFitGenerator(model)
    model.save('resnet_small_new.h5') 
    #saveCoreMLModel(model)

In [0]:
if __name__ == '__main__':
    # constants
    image_size = (224, 224)
    train_data_dir = '/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Small/Train'
    validation_data_dir = '/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Small/Val'
    nb_epoch = 1
    batch_size = 128
    num_classes = 5
    main()

In [0]:
model = load_model('/content/drive/My Drive/resnet_small_new.h5')

In [0]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_16 (Model)             (None, 100352)            23587712  
_________________________________________________________________
dense_22 (Dense)             (None, 512)               51380736  
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 2565      
Total params: 75,233,669
Trainable params: 51,645,957
Non-trainable params: 23,587,712
________________________________

In [0]:
compile_model(model)
modelFitGenerator(model)
model.save('/content/drive/My Drive/resnet_small_new.h5') 

In [0]:
from google.colab import files
files.download('/content/drive/My Drive/resnet_small_new.h5')